# 주제 분석

## 학습 목표

- 토픽 모형을 이용해 분석을 실시하고 해석할 수 있다

## 토픽 모형

- 2003년 David Blei, Andrew Ng, 그리고 Michael Jordan이 발표
- 여러 개의 텍스트에서 공통된 주제를 추출하는 기법

### 토픽 모형의 가정

- 하나의 텍스트에는 여러 가지 주제가 일정 비율로 포함되어 있다
- 주제마다 고유한 단어의 분포가 있다
- 텍스트의 단어 분포는 각 주제의 비율과 각 주제의 단어 분포에 따라 정해진다

## gensim

아나콘다에서는 다음 명령으로 설치

In [ ]:
!conda install -y gensim

colab에서는 `pip`로 설치

In [ ]:
!pip install gensim

## 토픽 모형 실습

### 데이터 불러오기

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('인공지능.csv', engine='python', index_col=0, encoding='utf8')

### WPM 불러오기

In [3]:
from subword_nmt.apply_bpe import BPE

In [4]:
with open('인공지능BPE.txt', encoding='utf8') as f:
    bpe = BPE(f, separator='~')

### 토크나이저

In [5]:
def tokenizer_wpm(text):
    tokens = bpe.process_line(text)
    tokens = tokens.split()
    return [t for t in tokens
            if (not t.endswith('~') and len(t) > 1) or len(t) > 2]

### TDM 만들기

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

In [7]:
cv = CountVectorizer(tokenizer=tokenizer_wpm, max_features=1000)

In [8]:
tdm = cv.fit_transform(df['article'])

In [9]:
words = cv.get_feature_names()

### TDM을 gensim의 형식으로 변환

In [12]:
from gensim.matutils import Sparse2Corpus

corpus = Sparse2Corpus(tdm.T)

C:\Users\user\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


### 토픽 모형 분석

In [13]:
from gensim.models.ldamodel import LdaModel

In [14]:
lda = LdaModel(corpus=corpus,
               id2word=dict(enumerate(words)),
               num_topics=10,
               passes=10,
               iterations=50,
               random_state=1234)

## 분석 결과 보기

0번 문서의 내용

In [15]:
df.loc[0, 'article']

'한솔인티큐브의 커뮤니케이션 브랜드 ‘아이작’은 옴니채널과 인공지능(AI) 기술 도입이 어려운 컨택센터를 위한 클라우드 서비스를 선보인다.\n         기업들은 빠르게 변화하는 시장 환경 속에서 고객의 니즈를 수용할 수 있는 유연한 플랫폼을 필요로 한다. 과거에는 전화라는 하나의 채널로 모이던 고객의 목소리가 최근에는 웹, 모바일 플랫폼 발전에 따라 다양화되고 있다. 다양한 채널은 시대에 따라 새로 추가되기도 하고 사라지기도 한다. 카카오톡이 문자 메시지(SMS) 사용량을 넘어선 것이 대표적인 예이다.\n         이렇듯 환경과 고객의 변화는 시간이 갈수록 가속화되고 예측이 불가하다. 앞으로 기업들은 어떻게 이러한 환경에 대응하여 최적의 고객 경험을 제공할 수 있을까?\n         과거와 같이 대규모 투자를 통한 인프라 구축 방식으로는 시장 변화 속도에 따라갈 수 없다. 특히 신기술 도입을 통한 서비스는 론칭 이후 시장의 검증 과정이 필수적이다. 고객의 소리에 따라 서비스를 지속적으로 개선해나가는 과정이 필요하다. AICC(인공지능 컨택센터)의 경우도 마찬가지이다.\n         인공지능 기술이 최근 각광받고 있음에도 불구하고 기업에서는 선뜻 AICC로의 전환을 결정하기가 어렵다. 기술의 장벽, 높은 초기 비용은 차치하더라도 그 효과성에 의문을 가지기 때문이다.\n         힌솔인티큐브의 아이작이 제안하는 클라우드 솔루션은 최근 가장 화두에 오르는 인공지능, 옴니채널, 클라우드를 모두 아우른 것이다. 서비스에 단순히 챗봇만을 추가하는 것이 아니라 고객의 다양한 채널에 AI를 접목할 수 있다. 전화 채널과 AI를 결합한 음성 봇은 물론, 문자, 메신저, 카카오톡 등과의 결합한 챗봇, 고객의 요청사항 처리를 자동화할 수 있는 RPA 등 AI와 상담원 간 유기적 협업 체제를 구현할 수 있는 솔루션이다.\n         또 챗봇, 음성 봇을 포함한 AI 상담 봇은 얼마나 정확하고 많은 데이터를 학습하는 가에 따라 서비스의 질이 달라진다. 기

### 문서 정보를 TDM에서 gensim 형식으로 변환

In [16]:
row = tdm[0]
doc = list(zip(row.indices, row.data))

### 문서의 주제 확인

- `(2, 0.79), (7, 0.20)`은 2번 주제 79%, 7번 주제 20%가 섞였다는 뜻

In [17]:
lda.get_document_topics(doc)

[(2, 0.79535955), (7, 0.2016031)]

### 특정 주제에서 자주 나오는 표현

In [18]:
lda.show_topic(2)

[('하고', 0.017391594),
 ('하는', 0.015492899),
 ('으로', 0.014047263),
 ('인공지능', 0.010525397),
 ('에서', 0.009355502),
 ('서비스를', 0.009160505),
 ('있다.', 0.00890011),
 ('기업~', 0.008296231),
 ('서비스', 0.008019856),
 ('통해', 0.0074179773)]

In [19]:
lda.show_topic(7)

[('에서', 0.018277612),
 ('데이터', 0.014089167),
 ('하는', 0.013355438),
 ('하고', 0.011498911),
 ('인공지능', 0.0112016415),
 ('한국~', 0.00956583),
 ('데이터~', 0.008560068),
 ('인공지능~', 0.008392587),
 ('으로', 0.008297168),
 ('의료~', 0.00781262)]

## 저장과 불러오기

저장

In [20]:
lda.save('lda_test.lda')

불러오기

In [21]:
lda2 = LdaModel.load('lda_test.lda') 